# Agile SEE Final Code 

In [1]:
pip install torch torchvision scikit-learn matplotlib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
from torchvision import datasets, transforms
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

# 1. Load and preprocess data
transform = transforms.ToTensor()
train_data = datasets.FashionMNIST(root='data', train=True, transform=transform, download=True)
test_data = datasets.FashionMNIST(root='data', train=False, transform=transform, download=True)

# Flatten data
X_train = train_data.data.view(-1, 28*28).numpy()
y_train = train_data.targets.numpy()
X_test = test_data.data.view(-1, 28*28).numpy()
y_test = test_data.targets.numpy()

# 2. Train model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# 3. Evaluate model
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# 4. Visualize predictions
labels_map = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
              'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

plt.figure(figsize=(10,4))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(X_test[i].reshape(28,28), cmap='gray')
    plt.title(f"Pred: {labels_map[y_pred[i]]}")
    plt.axis('off')
plt.tight_layout()
plt.show()


In [3]:
import torch
from torchvision import datasets, transforms
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt

# Load Fashion MNIST dataset 
transform = transforms.Compose([transforms.ToTensor()])
train_data = datasets.FashionMNIST(root="./data", train=True, download=True, transform=transform)
test_data = datasets.FashionMNIST(root="./data", train=False, download=True, transform=transform)

# Convert to NumPy
def dataset_to_numpy(dataset):
    X, y = [], []
    for img, label in dataset:
        X.append(img.view(-1).numpy())  
        y.append(label)
    return np.array(X), np.array(y)

X_train, y_train = dataset_to_numpy(train_data)
X_test, y_test = dataset_to_numpy(test_data)

print(" Dataset loaded.")
print("Training shape:", X_train.shape)
print("Test shape:", X_test.shape)


def evaluate_model(name, model, X_test, y_test, is_classifier=True):
    y_pred = model.predict(X_test)
    if not is_classifier:
        y_pred = np.round(y_pred).astype(int)
        y_pred = np.clip(y_pred, 0, 9)
    print(f"\n {name} Results:")
    print(f" Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")
    print(" Classification Report:\n", classification_report(y_test, y_pred))
    plt.figure(figsize=(8, 6))
    plt.imshow(confusion_matrix(y_test, y_pred), cmap='Blues')
    plt.title(f"{name} - Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.colorbar()
    plt.show()
    
    

# 1. Logistic Regression
log = LogisticRegression(max_iter=1000)
log.fit(X_train, y_train)
evaluate_model("Logistic Regression", log, X_test, y_test)

# 2. Linear Regression (treated as classification)
linear = LinearRegression()
linear.fit(X_train, y_train)3
evaluate_model("Linear Regression", linear, X_test, y_test, is_classifier=False)

# 3. Decision Tree
tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)
evaluate_model("Decision Tree", tree, X_test, y_test)

# 4. SVM
svm = SVC()
svm.fit(X_train[:10000], y_train[:10000])  # SVM is slow on large datasets
evaluate_model("SVM", svm, X_test, y_test)

# 5. Naive Bayes
nb = GaussianNB()
nb.fit(X_train, y_train)
evaluate_model("Naive Bayes", nb, X_test, y_test)

# 7. Sample prediction
labels_map = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
              'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
y_pred = log.predict(X_test[:10])

plt.figure(figsize=(10, 4))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(X_test[i].reshape(28, 28), cmap='gray')
    plt.title(f"True: {labels_map[y_test[i]]}\nPred: {labels_map[y_pred[i]]}")
    plt.axis('off')

plt.tight_layout()
plt.show()


SyntaxError: invalid syntax (3855811383.py, line 58)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. Load and preprocess the dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# 2. Define the Deep Neural Network (MLP)
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        return self.model(x)

model = MLP().to(device)

# 3. Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. Train the model
epochs = 100
for epoch in range(epochs):
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}")

# 5. Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total:.2f}%')
